In [1]:
from functools import reduce
import pandas as pd
import numpy as np
import mysql.connector

In [2]:


# declare a connection to the MySQL server 
# the hostname (or IP) of the server machine should be passed

db = mysql.connector.connect(
  user="root",
  password="R13251618a", 
  host="127.0.0.1",)


In [3]:
# creating the db cursor object
dbc = db.cursor()

In [4]:
database_to_use = "World_cup"

dbc.execute(f"USE {database_to_use};")

## Insrting into the races table

In [69]:
def insert_race(dbc, db, values_to_insert, columns_to_insert = None):
    """
    This function inserts values into the Races table
    Parameters:

        dbc_object: the cursor object obtained trough dbc.execute(DESCRIBE Races)
        values_to_insert: A tuple of values to insert corresponding to the columns_to_insert
        columns_to_insert: Tuple containing the columns to insert, if None all the columns except the Races_ID are inserted
        dbc: the cursor object
        db: the database object
    """
    dbc.execute("DESCRIBE Races;")
    dbc_object = dbc.fetchall()

    number_of_values = len(values_to_insert) # Number of values to insert

    formatted_string_to_insert = '%s,' * number_of_values # Preparing the formatting string format
    formatted_string_to_insert = formatted_string_to_insert[:-1] # Removing the last comma

    if columns_to_insert == None: # If no columns are specified, all the columns are inserted
        string_columns = map(lambda x: x[0], dbc_object) # Getting the columns names
        
        string_columns = list(string_columns)[1:] # Removing the first column (RaceID)
        
        string_columns = reduce(lambda x, y: x + y + ', ', string_columns, '') # Preparing the string to insert
        string_columns = string_columns[:-2] # Removing the last comma and space

    else:
        string_columns = str(columns_to_insert).replace("'", "") # Preparing the string to insert




    string_to_insert = f"INSERT INTO Races ({string_columns}) VALUES ({formatted_string_to_insert})" 
    
    val_ = [values_to_insert]
    dbc.executemany(string_to_insert, val_)
    db.commit()


In [74]:
dbc.execute(f'UPDATE Races SET Date =\'2022-11-29\' WHERE RaceID = 1;')
db.commit()


In [75]:
insert_race(dbc, db, ('22-23', 'Sprint', 'Kontiolahti', 'FIN', '2022-12-03', 'M', 10, 'E 1.4',\
     'E 1.9', 'E 1.2', 'E 2.2', 'Hard packed variable', 'Hard packed variable', 'Hard packed variable',\
         'Hard packed variable', -2.8,  -2.8, -2.5, -2.5, -3.3, -3.2, -3.2, -3.2, 91, 91, 91, 91, 15238, 37, 34, 510))

In [ ]:
def insert_results(dbc, db, path_csv, race_id):
    dbc.execute("DESCRIBE Results")
    dbc_object = dbc.fetchall()

    columns_df = pd.DataFrame(dbc_object) 
    columns_df = columns_df.loc[:,0].values
    string_columns = reduce(lambda x, y: x + y + ", ", columns_df, '')
    string_columns = string_columns[:-2]

    number_of_values = len(columns_df)

    formatted_string_to_insert = '%s,' * number_of_values # Preparing the formatting string format
    formatted_string_to_insert = formatted_string_to_insert[:-1] # Removing the last comma

    df = pd.read_csv(path_csv, sep="\t")
    
    df.insert(0, 'RaceID', race_id)
    df.Behind = df.Behind.apply(lambda x: x[1:] if type(x) == str else x)
    
    df.loc[0, 'Behind'] = '0'
    df.Behind = df.Behind.apply(lambda x: '00:' + x if x.count(':') == 1 else x) 
    df['Total Time'] = df['Total Time'].apply(lambda x: '00:' + x if x.count(':') == 1 else x)
    
    df.insert(3, 'FullName', df['Given Name'] + " " + df['Family\xa0Name'])
    for i in range(8, 12):
        df.insert(i, 'Mistakes_' + str(i-7), list(map(lambda x: x[(i-8)*2], np.array(df.Shootings))))
    df = df.drop(columns=['Shootings'])
    val_ = list(map(lambda x: tuple(x) , np.array(df)))
    results_statement = f"""INSERT INTO Results ({string_columns}) VALUES ({formatted_string_to_insert});"""

    
    dbc.executemany(results_statement, val_)
    db.commit()
    

In [ ]:
insert_results(dbc, db, res_, "../../../tmp.txt", 2)

In [ ]:
def insert_loop(dbc, db, path_csv, race_id, loop_id):
    #path_csv = '../../../tmp.txt'
    #race_id = 1
    #loop_id = '1'
    dbc.execute("DESCRIBE Loops;")
    dbc_object = dbc.fetchall()

    columns_df = pd.DataFrame(dbc_object) 
    columns_df = columns_df.loc[:,0].values
    string_columns = reduce(lambda x, y: x + y + ", ", columns_df, '')
    string_columns = string_columns[:-2]
    number_of_values = len(columns_df)
    formatted_string_to_insert = '%s,' * number_of_values # Preparing the formatting string format
    formatted_string_to_insert = formatted_string_to_insert[:-1] # Removing the last comma

    df = pd.read_csv(path_csv, sep="\t")
    df.insert(0, 'RaceID', race_id)
    df.insert(0, 'LoopID', loop_id)
    time_columns = list(df.columns)[list(df.columns).index('Cumulative Time'):]
    for column in time_columns:
        df[column] = df[column].apply(lambda x: '00:' + str(x) if str(x).count(':') == 1 else x)

    df.insert(4, 'FullName', df['Given Name'] + " " + df['Family\xa0Name'])

    val_ = list(map(lambda x: tuple(x) , np.array(df)))
    loops_statement = f"""INSERT INTO Loops ({string_columns}) VALUES ({formatted_string_to_insert});"""

    dbc.executemany(loops_statement, val_)
    db.commit()


In [ ]:
insert_loop(dbc, db, '../../../tmp.txt', race_id=1, loop_id='1')

In [ ]:
def insert_shooting(dbc, db, path_csv, race_id, n_ranges):
    dbc.execute("DESCRIBE Shooting;")
    dbc_object = dbc.fetchall()

    columns_df = pd.DataFrame(dbc_object) 
    columns_df = columns_df.loc[:,0].values
    string_columns = reduce(lambda x, y: x + y + ", ", columns_df, '')
    string_columns = string_columns[:-2]
    number_of_values = len(columns_df)
    formatted_string_to_insert = '%s,' * number_of_values # Preparing the formatting string format
    formatted_string_to_insert = formatted_string_to_insert[:-1] # Removing the last comma

    df = pd.read_csv(path_csv, sep="\t")
    formatted_string_to_insert
    df.insert(0, 'RaceID', race_id)
    df.insert(3, 'FullName', df['Given Name'] + " " + df['Family\xa0Name'])
    columns_to_change =  np.array(df.columns[list(df.columns).index('Lane'):])
    filt_column_change = columns_to_change[np.logical_or(np.arange(len(columns_to_change)) % 3 == 0,\
        np.arange(len(columns_to_change)) % 3 == 2)]
    columns_final = [[f'Lane{i}', f'Time{i}'] for i in range(1, n_ranges + 1)]
    columns_final_arr = np.array(columns_to_change)
    columns_final_arr[np.logical_or(np.arange(len(columns_to_change)) % 3 == 0,\
        np.arange(len(columns_to_change)) % 3 == 2)] = reduce(lambda x, y: x + y, columns_final, [])
    df.columns = np.concatenate([np.array(df.columns[:list(df.columns).index('Lane')]), columns_final_arr])
    for shooting in range(1, n_ranges + 1):
        df[f'Shooting {shooting}'] = df[f'Shooting {shooting}'].apply(lambda x: reduce(lambda x, y: x + ' ' + y,\
            list(filter(lambda x: x!='', map(lambda x: x.strip(), x.split('\xa0'))))))
        
    val_ = list(map(lambda x: tuple(x) , np.array(df)))
    loops_statement = f"""INSERT INTO Shooting ({string_columns}) VALUES ({formatted_string_to_insert});"""

    dbc.executemany(loops_statement, val_)
    db.commit()

In [ ]:
insert_shooting(dbc, db, '../../../tmp.txt', race_id = 1, n_ranges = 4)

In [ ]:
def insert_analysis(dbc, db, path_csv, race_id):

    dbc.execute("DESCRIBE Analysis;")
    dbc_object = dbc.fetchall()

    columns_df = pd.DataFrame(dbc_object) 
    columns_df = columns_df.loc[:,0].values
    string_columns = reduce(lambda x, y: x + y + ", ", columns_df, '')
    string_columns = string_columns[:-2]
    number_of_values = len(columns_df)
    formatted_string_to_insert = '%s,' * number_of_values # Preparing the formatting string format
    formatted_string_to_insert = formatted_string_to_insert[:-1] # Removing the last comma
    formatted_string_to_insert
    df = pd.read_csv(path_csv, sep="\t")
    df.insert(0, 'RaceID', race_id)
    df.insert(3, 'FullName', df['Given Name'] + " " + df['Family\xa0Name'])
    df['Total Time'] = df['Total Time'].apply(lambda x: '00:' + str(x) if str(x).count(':') == 1 else x)
    
    val_ = list(map(lambda x: tuple(x) , np.array(df)))
    analysis_statement = f"""INSERT INTO Analysis ({string_columns}) VALUES ({formatted_string_to_insert});"""

    dbc.executemany(analysis_statement, val_)
    db.commit()


In [ ]:
insert_analysis(dbc, db, path_csv = '../../../tmp.txt', race_id = 1)

In [ ]:
def insert_split_time(dbc, db, path_csv, race_id, km):

    dbc.execute("DESCRIBE SplitTimes;")
    dbc_object = dbc.fetchall()

    columns_df = pd.DataFrame(dbc_object) 
    columns_df = columns_df.loc[:,0].values
    string_columns = reduce(lambda x, y: x + y + ", ", columns_df, '')
    string_columns = string_columns[:-2]
    number_of_values = len(columns_df)
    formatted_string_to_insert = '%s,' * number_of_values # Preparing the formatting string format
    formatted_string_to_insert = formatted_string_to_insert[:-1] # Removing the last comma
    df = pd.read_csv(path_csv, sep="\t")
    df.insert(0, 'RaceID', race_id)
    df.insert(3, 'km', km)
    df.insert(4, 'FullName', df['Given Name'] + " " + df['Family\xa0Name'])
    time_columns = ['Day Time', 'Sector Time', 'Net Time', 'Behind']
    for col in time_columns:
        df[col] = df[col].apply(lambda x: '00:' + str(x) if str(x).count(':') == 1 else x)


    val_ = list(map(lambda x: tuple(x) , np.array(df)))
    split_times_statement = f"""INSERT INTO SplitTimes ({string_columns}) VALUES ({formatted_string_to_insert});"""

    dbc.executemany(split_times_statement, val_)
    db.commit()

In [ ]:
insert_split_time(dbc, db, '../../../tmp.txt', race_id = 1, km = 1.0)

In [76]:
dbc.close()

True

In [77]:
db.close()